In [0]:
%sql
WITH data_ref AS (
    SELECT max(order_purchase_timestamp) AS ref_date
    FROM workspace.olist_silver.pedidos_silver
),
rfv AS(
      SELECT 
          p.customer_id,

          DATEDIFF(
            (SELECT ref_date FROM data_ref),
             max(p.order_purchase_timestamp)
             ) AS recency,

          COUNT(DISTINCT p.order_id) AS frequency,

          SUM(pg.payment_value) AS monetary_value

            
      FROM workspace.olist_silver.pedidos_silver p 
      LEFT JOIN workspace.olist_silver.pagamentos_silver pg
            ON  p.order_id = pg.order_id

      WHERE p.order_status = 'delivered'
      GROUP BY p.customer_id
      )
SELECT * FROM rfv;